In [65]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize
import ast

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

In [66]:
# api_key 저장
api_key = 'RGAPI-b2bd2659-5fb4-4be0-821a-e3af369cc138' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

##### 앞에서 만든 게임 아이디 테이블 불러오기

In [67]:
gameId_df = pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/tier_and_gameId.csv')

gameId_df.head(3)   

,tier,gameId
0,GRANDMASTER,KR_7004319615
1,GRANDMASTER,KR_7004261207
2,GRANDMASTER,KR_7004219924


#### 게임 아이디 테이블의 모든 게임 결과 데이터 불러오기

##### 하나의 게임 아이디로부터 게임 결과 데이터를 불러오는 쿼리 작성 테스트

In [77]:
gameId = 'KR_7003200600'

# gameId 를 통해 game result data 를 불러오는 API URL 설정
game_result_url = "https://Asia.api.riotgames.com/tft/" + f"match/v1/matches/{gameId}"

# 조회한 결과를 game_result 에 저장
game_result = requests.get(game_result_url, headers=request_header).json()

# 게임 데이터 중 실질적으로 중요한 정보를 담고있는 info 컬럼을 데이터 프레임으로 저장
game_result_df = pd.DataFrame(game_result['info'])

# 반환된 게임 결과 데이터 프레임의 gameId 컬럼에 KR_ 를 붙여주는 것으로 tier_gameId 테이블과 컬럼값을 동일하게
game_result_df['gameId'] = 'KR_' + game_result_df['gameId'].astype(str)

# 필요없는 컬럼 drop
game_result_df = game_result_df[['gameId', 'game_datetime', 'game_length', 'game_version', 'participants']]

In [ ]:
game_result_df.head(1)

,gameId,game_datetime,game_length,game_version,participants
0,KR_7004319615,1711529215362,2305.195557,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,"{'augments': ['TFT6_Augment_ThreesCompany', 'TFT9_Augment_HealingOrbsI', 'TFT9_Augment_OldMansWalkingStick'], 'companion': {'content_ID': 'a9d00460-7cd2-4f6a-8669-21d6eb2838e9', 'item_ID': 74002, 'skin_ID': 2, 'species': 'PetChibiIrelia'}, 'gold_left': 0, 'last_round': 38, 'level': 9, 'missions': {'Assists': 0, 'DamageDealt': 1, 'DamageDealtToObjectives': 181582, 'DamageDealtToTurrets': 2, 'DamageTaken': 0, 'Deaths': 15, 'DoubleKills': 0, 'GoldEarned': 0, 'GoldSpent': 0, 'InhibitorsDestroyed': 0, 'KillingSprees': 17, 'Kills': 0, 'LargestKillingSpree': 57948, 'LargestMultiKill': 146675, 'MagicDamageDealt': 0, 'MagicDamageDealtToChampions': 0, 'MagicDamageTaken': 0, 'NeutralMinionsKilledTeamJungle': 0, 'PentaKills': 1, 'PhysicalDamageDealt': 0, 'PhysicalDamageDealtToChampions': 30, 'PhysicalDamageTaken': 3, 'PlayerScore0': 3, 'PlayerScore1': 8, 'PlayerScore10': 0, 'PlayerScore11': 0, 'PlayerScore2': 215, 'PlayerScore3': 0, 'PlayerScore4': 1, 'PlayerScore5': 1, 'PlayerScore6': 10, 'PlayerScore9': 0, 'QuadraKills': 0, 'Spell1Casts': 0, 'Spell2Casts': 1, 'Spell3Casts': 16, 'Spell4Casts': 11234, 'SummonerSpell1Casts': 0, 'TimeCCOthers': 2, 'TotalDamageDealtToChampions': 0, 'TotalMinionsKilled': 0, 'TripleKills': 0, 'TrueDamageDealt': 1, 'TrueDamageDealtToChampions': 62, 'TrueDamageTaken': 8, 'UnrealKills': 1, 'VisionScore': 1, 'WardsKilled': 61}, 'placement': 3, 'players_eliminated': 0, 'puuid': 'iCcuYtTqqr1BKUjBwIDfE1FWhLj4Nog495kIOrxNqEfWq6iboiWi8epQ443aAIBGsy4ej3pdRyrfuQ', 'time_eliminated': 2150.0068359375, 'total_damage_to_players': 154, 'traits': [{'name': 'TFT11_Altruist', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Arcanist', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Bruiser', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Duelist', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_Fortune', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Ghostly', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Great', 'num_units': 1, 'style': 5, 'tier_current': 1, 'tier_total': 1}, {'name': 'TFT11_Heavenly', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 6}, {'name': 'TFT11_Sage', 'num_units': 3, 'style': 2, 'tier_current': 2, 'tier_total': 4}, {'name': 'TFT11_Storyweaver', 'num_units': 7, 'style': 3, 'tier_current': 3, 'tier_total': 4}, {'name': 'TFT11_Warden', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 3}], 'units': [{'character_id': 'TFT11_Garen', 'itemNames': [], 'name': '', 'rarity': 0, 'tier': 2}, {'character_id': 'TFT11_Riven', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Zyra', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Illaoi', 'itemNames': ['TFT_Item_GargoyleStoneplate', 'TFT_Item_DragonsClaw', 'TFT_Item_RedBuff'], 'name': '', 'rarity': 2, 'tier': 2}, {'character_id': 'TFT11_Zoe', 'itemNames': ['TFT_Item_Leviathan', 'TFT_Item_Morellonomicon'], 'name': '', 'rarity': 2, 'tier': 2}, {'character_id': 'TFT11_Morgana', 'itemNames': ['TFT11_Item_StoryweaverEmblemItem', 'TFT_Item_SpearOfShojin', 'TFT_Item_JeweledGauntlet'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Galio', 'itemNames': [], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Irelia', 'itemNames': ['TFT_Item_GuinsoosRageblade', 'TFT_Item_RunaansHurricane', 'TFT_Item_Bloodthirster'], 'name': '', 'rarity': 6, 'tier': 2}, {'character_id': 'TFT11_WuKong', 'itemNames': [], 'name': '', 'rarity': 6, 'tier': 1}, {'character_id': 'TFT11_Kayle', 'itemNames': ['TFT11_Storyweaver7_AS', 'TFT11_Storyweaver5_AS', 'TFT11_Storyweaver3_Support'], 'name': '', 'rarity': 9, 'tier': 3}]}"


##### 모든 게임 아이디에 대한 게임 결과 데이터를 조회하여 하나의 테이블로 만들기

In [75]:
# gameId 목록 추출
game_ids = list(gameId_df['gameId'])

# 빈 데이터 프레임 생성 (초기에는 데이터 없음)
game_results_df = pd.DataFrame()

# 각 게임 아이디에 대한 게임 결과 정보 조회 및 저장
for game_id in game_ids:
  game_result_url = "https://Asia.api.riotgames.com/tft/" + f"match/v1/matches/{game_id}"
  game_result = requests.get(game_result_url, headers=request_header).json()

  # Check if 'info' key exists
  if 'info' in game_result:
    # Create a temporary DataFrame from game_result['info']
    temp_df = pd.DataFrame(game_result['info'])
    # Concatenate the temporary DataFrame with game_results_df
    game_results_df = pd.concat([game_results_df, temp_df], ignore_index=True)
  else:
    # Handle missing information (log a warning or assign a default value)
    print(f"WARNING: 'info' key not found for game ID: {game_id}")

# 필요없는 컬럼 drop
game_results_df = game_results_df[['gameId', 'game_datetime', 'game_length', 'game_version', 'participants']]

# gameId 컬럼에 'KR_' 추가
game_results_df['gameId'] = 'KR_' + game_results_df['gameId'].astype(str)

# 결과 확인
print(game_results_df.head())

KeyboardInterrupt: 

##### game_result_df 로부터 필요한 컬럼의 json 형식을 normalize 하여 테이블화

In [ ]:
# 새로운 데이터프레임 생성
paricipants_df = game_result_df[['gameId', 'game_datetime', 'game_length', 'game_version']].copy()

# 'participants' 컬럼의 데이터에서 'puuid' 값을 추출하여 새로운 열 생성

paricipants_df['puuid'] = game_result_df['participants'].apply(lambda x: x['puuid']) # 유저 식별자

# 'placement'

paricipants_df['placement'] = game_result_df['participants'].apply(lambda x: x['placement']) # 최종 등수

# 'last_round'

paricipants_df['last_round'] = game_result_df['participants'].apply(lambda x: x['last_round']) # 버틴 라운드 수

# 'level'

paricipants_df['level'] = game_result_df['participants'].apply(lambda x: x['level']) # 경기 종료 시 플레이어 레벨

# 'Augments'

paricipants_df['augments'] = game_result_df['participants'].apply(lambda x: x['augments']) # 경기 종료 시 보유한 증강체

# 'traits'

paricipants_df['traits'] = game_result_df['participants'].apply(lambda x: x['traits']) # 경기 종료 시 활성화된 시너지

# 'units'

paricipants_df['units'] = game_result_df['participants'].apply(lambda x: x['units']) # 경기 종료시 가지고 있는 유닛, 장착된 아이템

# 결과 확인
paricipants_df.head(1)

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,augments,traits,units
0,KR_7004319615,1711529215362,2305.195557,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,iCcuYtTqqr1BKUjBwIDfE1FWhLj4Nog495kIOrxNqEfWq6iboiWi8epQ443aAIBGsy4ej3pdRyrfuQ,3,38,9,"[TFT6_Augment_ThreesCompany, TFT9_Augment_HealingOrbsI, TFT9_Augment_OldMansWalkingStick]","[{'name': 'TFT11_Altruist', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Arcanist', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Bruiser', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Duelist', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_Fortune', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Ghostly', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Great', 'num_units': 1, 'style': 5, 'tier_current': 1, 'tier_total': 1}, {'name': 'TFT11_Heavenly', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 6}, {'name': 'TFT11_Sage', 'num_units': 3, 'style': 2, 'tier_current': 2, 'tier_total': 4}, {'name': 'TFT11_Storyweaver', 'num_units': 7, 'style': 3, 'tier_current': 3, 'tier_total': 4}, {'name': 'TFT11_Warden', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 3}]","[{'character_id': 'TFT11_Garen', 'itemNames': [], 'name': '', 'rarity': 0, 'tier': 2}, {'character_id': 'TFT11_Riven', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Zyra', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Illaoi', 'itemNames': ['TFT_Item_GargoyleStoneplate', 'TFT_Item_DragonsClaw', 'TFT_Item_RedBuff'], 'name': '', 'rarity': 2, 'tier': 2}, {'character_id': 'TFT11_Zoe', 'itemNames': ['TFT_Item_Leviathan', 'TFT_Item_Morellonomicon'], 'name': '', 'rarity': 2, 'tier': 2}, {'character_id': 'TFT11_Morgana', 'itemNames': ['TFT11_Item_StoryweaverEmblemItem', 'TFT_Item_SpearOfShojin', 'TFT_Item_JeweledGauntlet'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Galio', 'itemNames': [], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Irelia', 'itemNames': ['TFT_Item_GuinsoosRageblade', 'TFT_Item_RunaansHurricane', 'TFT_Item_Bloodthirster'], 'name': '', 'rarity': 6, 'tier': 2}, {'character_id': 'TFT11_WuKong', 'itemNames': [], 'name': '', 'rarity': 6, 'tier': 1}, {'character_id': 'TFT11_Kayle', 'itemNames': ['TFT11_Storyweaver7_AS', 'TFT11_Storyweaver5_AS', 'TFT11_Storyweaver3_Support'], 'name': '', 'rarity': 9, 'tier': 3}]"
